In [4]:
from sklearn.ensemble import VotingClassifier
from tensorflow.keras.models import load_model


In [5]:
#load model
model = load_model('model02.h5')
# summarize model.
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 16)                480       
_________________________________________________________________
dense_8 (Dense)              (None, 24)                408       
_________________________________________________________________
dropout_1 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 20)                500       
_________________________________________________________________
dense_10 (Dense)             (None, 24)                504       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 25        
Total params: 1,917
Trainable params: 1,917
Non-trainable params: 0
____________________________________________________

In [7]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [9]:
df = pd.read_csv('creditcard.csv')
scaler = StandardScaler()
df['NormalizedAmount'] = scaler.fit_transform(df['Amount'].values.reshape(-1, 1))
df = df.drop(['Amount', 'Time'], axis = 1)
y = df['Class']
X = df.drop(['Class'], axis = 1)

In [10]:
from sklearn.model_selection import train_test_split 
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)


In [12]:
from imblearn.over_sampling import SMOTE
X_resample1, y_resample1 = SMOTE().fit_resample(trainX, trainy)

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_resample1, y_resample1) 

In [15]:
yhat_probs = knn.predict(testX)
# predict crisp classes for test set
yhat_classes = knn.predict_classes(testX)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-7f60e9504bef>", line 1, in <module>
    yhat_probs = knn.predict(testX)
  File "c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\neighbors\_classification.py", line 197, in predict
    neigh_dist, neigh_ind = self.kneighbors(X)
  File "c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\neighbors\_base.py", line 708, in kneighbors
    **kwds))
  File "c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\pairwise.py", line 1633, in pairwise_distances_chunked
    D_chunk = reduce_func(D_chunk, sl.start)
  File "c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\sklearn\neighbors\_base.py", line 582, in _kneighbors_reduce_func
    n

KeyboardInterrupt: 

In [16]:
# save the model to disk
filename = 'knn_model.sav'
pickle.dump(knn, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

In [17]:
filename1='Iforest_model.sav'
Iforest = pickle.load(open(filename1, 'rb'))

In [21]:
from sklearn.ensemble import VotingClassifier
#create a dictionary of our models
#estimators=[('knn', knn), ('Iforest', Iforest), ('SequentialDL', model)]
estimators=[('knn', knn),('SequentialDL', model)]
#create our voting classifier, inputting our models
ensemble = VotingClassifier(estimators, voting='soft')

In [ ]:
#fit model to training data
ensemble.fit(trainX, trainy)



In [ ]:
#test our model on the test data
ensemble.score(testX, testy)